# Notebook: MLFlow Registry Management (Local & Production) - Version multi-modèles

---
# 1 vérification de l'occupation du port (MLflow déjà en route)

In [1]:
import subprocess
import socket

In [2]:
def check_and_launch_mlflow(port=5000):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', port))
    if result == 0:
        print(f"✅ MLFlow Tracking Server déjà actif sur http://127.0.0.1:{port}")
    else:
        print(f"🚀 Lancement du MLFlow Tracking Server sur le port {port}...")
        subprocess.Popen(
            ["mlflow", "server", "--backend-store-uri", "./mlruns", "--default-artifact-root", "./mlruns", "--host", "127.0.0.1", "--port", str(port)],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
    sock.close()

In [3]:
check_and_launch_mlflow()

✅ MLFlow Tracking Server déjà actif sur http://127.0.0.1:5000


# 1 -  Connexion au Tracking Server

In [4]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.pyfunc import load_model

## 1.1 - MLFlow Tracking URI (modifiable pour prod/local)

In [5]:
mlflow_url = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(mlflow_url)

In [6]:
print(f"\U0001F7E2 Connecté à MLFlow Tracking Server : {mlflow_url}")
print(f"MLFlow Version : {mlflow.__version__}")

🟢 Connecté à MLFlow Tracking Server : http://127.0.0.1:5000
MLFlow Version : 2.21.0


# 2 - Listing des modèles enregistrés

In [7]:
client = MlflowClient()
# models = client.list_registered_models()

In [8]:
models = client.search_registered_models()

In [9]:
print(f"\n📋 {len(models)} modèles trouvés dans le Model Registry :\n")
for model in models:
    print(f"🔹 Nom du modèle : {model.name}")
    print(f"   Dernière(s) versions : {[v.version for v in model.latest_versions]}")
    print(f"   Description : {model.description if model.description else 'Aucune'}")
    print("-" * 50)


📋 4 modèles trouvés dans le Model Registry :

🔹 Nom du modèle : sentiment_model_lgbm
   Dernière(s) versions : ['1']
   Description : Aucune
--------------------------------------------------
🔹 Nom du modèle : sentiment_model_logreg
   Dernière(s) versions : ['1']
   Description : Aucune
--------------------------------------------------
🔹 Nom du modèle : sentiment_model_lstm
   Dernière(s) versions : ['1']
   Description : Aucune
--------------------------------------------------
🔹 Nom du modèle : sentiment_model_rf
   Dernière(s) versions : ['1']
   Description : Aucune
--------------------------------------------------


# 3 - Récupération des détails et promotion éventuelle pour chaque modèle

In [10]:
for model in models:
    print(f"\n📊 Détails pour le modèle : {model.name}")

    for version_info in model.latest_versions:
        print(f"   ➤ Version : {version_info.version}")
        print(f"     Current Stage : {version_info.current_stage}")
        print(f"     Run ID : {version_info.run_id}")

        # Promotion en Production si pas encore fait
        if version_info.current_stage != "Production":
            client.transition_model_version_stage(
                name=model.name,
                version=version_info.version,
                stage="Production"
            )
            print(f"🚀 Version {version_info.version} promue en Production.")
        else:
            print(f"✅ Version {version_info.version} déjà en Production.")

        # Téléchargement dynamique du modèle depuis Registry
        model_uri = f"models:/{model.name}/Production"
        try:
            model_loaded = load_model(model_uri)
            print(f"✅ Modèle {model.name} chargé depuis Registry : {model_uri}")
        except Exception as e:
            print(f"❌ Impossible de charger {model.name} : {e}")

    print("-" * 50)

print("\n🎯 Traitement de tous les modèles terminé.")


📊 Détails pour le modèle : sentiment_model_lgbm
   ➤ Version : 1
     Current Stage : None
     Run ID : ba17c2acf2784d7aa195001132ba0e2e
🚀 Version 1 promue en Production.


C:\Users\motar\AppData\Local\Temp\ipykernel_4208\3089628948.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


✅ Modèle sentiment_model_lgbm chargé depuis Registry : models:/sentiment_model_lgbm/Production
--------------------------------------------------

📊 Détails pour le modèle : sentiment_model_logreg
   ➤ Version : 1
     Current Stage : None
     Run ID : 75fa370913e04d5daba4da11f4e802fa
🚀 Version 1 promue en Production.
✅ Modèle sentiment_model_logreg chargé depuis Registry : models:/sentiment_model_logreg/Production
--------------------------------------------------

📊 Détails pour le modèle : sentiment_model_lstm
   ➤ Version : 1
     Current Stage : None
     Run ID : cb6658326fb0409cb6d0f11b2f4ce70e
🚀 Version 1 promue en Production.


C:\Users\motar\AppData\Local\Temp\ipykernel_4208\3089628948.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\motar\AppData\Local\Temp\ipykernel_4208\3089628948.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
c:\Users\motar\Desktop\1-openclassrooms\AI_Engineer\1-projets\P07\P7_sentiment_MLOps\env_p7_MLOps\

✅ Modèle sentiment_model_lstm chargé depuis Registry : models:/sentiment_model_lstm/Production
--------------------------------------------------

📊 Détails pour le modèle : sentiment_model_rf
   ➤ Version : 1
     Current Stage : None
     Run ID : 19f259f8b0374b68a617af58a59a0f5c
🚀 Version 1 promue en Production.
✅ Modèle sentiment_model_rf chargé depuis Registry : models:/sentiment_model_rf/Production
--------------------------------------------------

🎯 Traitement de tous les modèles terminé.


# 4 - Aide à l'intégration dans API ou CI/CD plus tard

In [11]:
print("\n🔽 Bonnes pratiques recommandées :")
print("- Versionner systématiquement tous les modèles via le Registry.")
print("- Toujours charger les modèles dynamiquement depuis le Registry dans vos API.")
print("- Gérer la promotion des modèles sans toucher au code API (via MLflowClient ou MLFlow UI).")
print("- Possibilité d'automatiser ce notebook dans un pipeline CI/CD (GitHub Actions, etc.).")


🔽 Bonnes pratiques recommandées :
- Versionner systématiquement tous les modèles via le Registry.
- Toujours charger les modèles dynamiquement depuis le Registry dans vos API.
- Gérer la promotion des modèles sans toucher au code API (via MLflowClient ou MLFlow UI).
- Possibilité d'automatiser ce notebook dans un pipeline CI/CD (GitHub Actions, etc.).
